<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---


<p align="center"><h1 align="center">California Housing Model Submission Guide

##### <p align="center">*Source: Sklearn [California Housing Dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)* 

---
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data  with Sklearn Column Transformer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard

**Objective:** Predict median house value for California districts, expressed in hundreds of thousands of dollars

**Data**: 1990 Census attributes by Block Group. 
(A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data. A block group typically has a population of 600 to 3,000 people.) 

**Features**
* **MedInc** median income in block group
* **HouseAge** median house age in block group
* **AveRooms** average number of rooms per household
* **AveBedrms** average number of bedrooms per household
* **Population** block group population
* **AveOccup** average number of household members
* **Latitude** block group latitude
* **Longitude** block group longitude

**Target**
*   Median house value for California districts, expressed in hundreds of thousands of dollars ($100,000)

## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [28]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/ca_housing_competition_data-repository:latest') 


Data downloaded successfully.


In [29]:
# Load data into X_train, y_train, and X_test objects
import pandas as pd

X_train = pd.read_csv("ca_housing_competition_data/X_train.csv")
y_train = pd.read_csv("ca_housing_competition_data/y_train.csv", squeeze=True)

X_test=pd.read_csv("ca_housing_competition_data/X_test.csv")

X_train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.2596,33.0,5.017657,1.006421,2300.0,3.691814,32.71,-117.03
1,3.8125,49.0,4.473545,1.041005,1314.0,1.738095,33.77,-118.16
2,4.1563,4.0,5.645833,0.985119,915.0,2.723214,34.66,-120.48
3,1.9425,36.0,4.002817,1.033803,1418.0,3.994366,32.69,-117.11
4,3.5542,43.0,6.268421,1.134211,874.0,2.300000,36.78,-119.80


In [30]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(16512, 8)
(4128, 8)
(16512,)


##2.   Preprocess data using Sklearn Column Transformer / Write and Save Preprocessor function


In [5]:
# In this case we use Sklearn's Column transformer in our preprocessor function

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Create the preprocessing pipeline for both numeric data.
numeric_features=X_train.columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# final preprocessor object set up with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features)])

#Fit your preprocessor object
preprocess=preprocessor.fit(X_train) 

In [6]:
# Write function to transform data with preprocessor

def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [7]:
# check shape of X data 
preprocessor(X_train).shape

(16512, 8)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras

feature_count=preprocessor(X_train).shape[1] #count features in input data

model = Sequential()
model.add(Dense(64, input_dim=feature_count, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, kernel_initializer='normal')) 
                                            
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Fitting the NN to the Training set
model.fit(preprocessor(X_train), y_train, 
               batch_size = 60, 
               epochs = 50, validation_split=0.35)  

#### Save preprocessor function to local "preprocessor.zip" file

In [10]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [11]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [12]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://hcvbryu1a3.execute-api.us-east-1.amazonaws.com/prod/m"
#This is the unique rest api that powers this CA Housing Prediction Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [13]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [14]:
#Submit Model 1: 

#-- Generate predicted values (a list of predicted car prices) (Model 1)
predicted_values = model.predict(preprocessor(X_test))

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=predicted_values)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 6

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1394


In [15]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,mse,rmse,mae,r2,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,relu_act,loss,optimizer,model_config,memory_size,username,version
0,0.27,0.52,0.35,0.79,keras,False,True,Sequential,4.000000,8961.000000,4.000000,3.000000,str,Adam,"{'name': 'sequential', 'layers...",1390304.000000,gstreett,6
1,0.40,0.63,0.45,0.70,sklearn,False,False,GradientBoostingRegressor,nan,nan,nan,nan,nan,nan,"{'alpha': 0.9, 'ccp_alpha': 0....",nan,gstreett,5
2,0.46,0.68,0.49,0.65,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,gstreett,4
3,0.56,0.75,0.53,0.58,sklearn,False,False,LinearRegression,nan,8.000000,nan,nan,nan,nan,"{'copy_X': True, 'fit_intercep...",nan,gstreett,1
4,0.60,0.78,0.58,0.54,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,gstreett,2
5,0.73,0.86,0.65,0.44,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,gstreett,3


## 5. Repeat submission process to improve place on leaderboard


In [ ]:
# Create model 2  -- Model with Dropout regularization
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization

feature_count=preprocessor(X_train).shape[1] #count features in input data

model_2 = Sequential()
model_2.add(Dense(64, input_dim=feature_count))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(Dense(64))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(Dense(64))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(Dense(1, kernel_initializer='normal')) 
                                            
# Compile model
model_2.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Fitting the NN to the Training set
model_2.fit(preprocessor(X_train), y_train, 
               batch_size = 60, 
               epochs = 60, validation_split=0.35)       

In [17]:
# Save Model 2 to .onnx file

onnx_model = model_to_onnx(model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

# Save model to local .onnx file
with open("model_2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString()) 

In [18]:
# Submit Model 2

#-- Generate predicted y values (Model 2)
prediction_labels = model_2.predict(preprocessor(X_test))

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model_2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 7

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1394


In [19]:
# Compare two or more models
data=mycompetition.compare_models([6, 7], verbose=1)
mycompetition.stylize_compare(data)

,Model_6_Layer,Model_6_Shape,Model_6_Params,Model_7_Layer,Model_7_Shape,Model_7_Params
0,Dense,"[None, 64]",576.000000,Dense,"[None, 64]",576
1,Dense,"[None, 64]",4160.000000,BatchNormalization,"[None, 64]",256
2,Dense,"[None, 64]",4160.000000,Activation,"[None, 64]",0
3,Dense,"[None, 1]",65.000000,Dense,"[None, 64]",4160
4,None,None,nan,BatchNormalization,"[None, 64]",256
5,None,None,nan,Activation,"[None, 64]",0
6,None,None,nan,Dense,"[None, 64]",4160
7,None,None,nan,BatchNormalization,"[None, 64]",256
8,None,None,nan,Activation,"[None, 64]",0
9,None,None,nan,Dense,"[None, 1]",65


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [21]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.regularizers import l1, l2, l1_l2
import keras_tuner as kt

feature_count=preprocessor(X_train).shape[1] #count features in input data

#Define model structure & parameter search space with function
def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(64, input_dim=feature_count, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(units=hp.Int("units", min_value=32, max_value=512, step=32), #range 32-512 inclusive, minimum step between tested values is 32
                    activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(
        loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model                                          

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="mean_squared_error", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=3, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(x_train_split), y_train_split, epochs=2, validation_data=(preprocessor(x_val), y_val))

Trial 3 Complete [00h 00m 17s]
mean_squared_error: 0.4149565001328786

Best mean_squared_error So Far: 0.41092586517333984
Total elapsed time: 00h 01m 03s


In [23]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=preprocessor(X_train), y=y_train, epochs=5)

Epoch 1/5
516/516 [==============================] - 2s 3ms/step - loss: 1.5300 - mean_squared_error: 0.6819
Epoch 2/5
516/516 [==============================] - 2s 3ms/step - loss: 0.7813 - mean_squared_error: 0.4114
Epoch 3/5
516/516 [==============================] - 2s 3ms/step - loss: 0.6218 - mean_squared_error: 0.3925
Epoch 4/5
516/516 [==============================] - 2s 3ms/step - loss: 0.5475 - mean_squared_error: 0.3839
Epoch 5/5
516/516 [==============================] - 2s 4ms/step - loss: 0.5024 - mean_squared_error: 0.3734


In [24]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [25]:
#Submit Model 3: 

#-- Generate predicted values
prediction_labels = tuned_model.predict(preprocessor(X_test))

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 8

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1394


In [26]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,mse,rmse,mae,r2,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,batchnormalization_layers,relu_act,loss,optimizer,model_config,memory_size,username,version
0,0.27,0.52,0.35,0.79,keras,False,True,Sequential,4.000000,8961.000000,4.000000,nan,3.000000,str,Adam,"{'name': 'sequential', 'layers...",1390304.000000,gstreett,6
1,0.37,0.61,0.43,0.72,keras,False,True,Sequential,4.000000,34305.000000,4.000000,nan,3.000000,str,Adam,"{'name': 'sequential_1', 'laye...",1499736.000000,gstreett,8
2,0.40,0.63,0.45,0.70,sklearn,False,False,GradientBoostingRegressor,nan,nan,nan,nan,nan,nan,nan,"{'alpha': 0.9, 'ccp_alpha': 0....",nan,gstreett,5
3,0.46,0.68,0.49,0.65,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,gstreett,4
4,0.51,0.71,0.46,0.61,keras,False,True,Sequential,7.000000,9729.000000,4.000000,3.000000,3.000000,str,Adam,"{'name': 'sequential_1', 'laye...",2919448.000000,gstreett,7
5,0.56,0.75,0.53,0.58,sklearn,False,False,LinearRegression,nan,8.000000,nan,nan,nan,nan,nan,"{'copy_X': True, 'fit_intercep...",nan,gstreett,1
6,0.60,0.78,0.58,0.54,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,gstreett,2
7,0.73,0.86,0.65,0.44,sklearn,False,False,RandomForestRegressor,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,gstreett,3


In [27]:
# Compare two or more models
data=mycompetition.compare_models([6, 7, 8], verbose=1)
mycompetition.stylize_compare(data)

,Model_6_Layer,Model_6_Shape,Model_6_Params,Model_7_Layer,Model_7_Shape,Model_7_Params,Model_8_Layer,Model_8_Shape,Model_8_Params
0,Dense,"[None, 64]",576.000000,Dense,"[None, 64]",576,Dense,"[None, 64]",576.000000
1,Dense,"[None, 64]",4160.000000,BatchNormalization,"[None, 64]",256,Dense,"[None, 64]",4160.000000
2,Dense,"[None, 64]",4160.000000,Activation,"[None, 64]",0,Dense,"[None, 448]",29120.000000
3,Dense,"[None, 1]",65.000000,Dense,"[None, 64]",4160,Dense,"[None, 1]",449.000000
4,None,None,nan,BatchNormalization,"[None, 64]",256,None,None,nan
5,None,None,nan,Activation,"[None, 64]",0,None,None,nan
6,None,None,nan,Dense,"[None, 64]",4160,None,None,nan
7,None,None,nan,BatchNormalization,"[None, 64]",256,None,None,nan
8,None,None,nan,Activation,"[None, 64]",0,None,None,nan
9,None,None,nan,Dense,"[None, 1]",65,None,None,nan
